## LangChain의 SQLDatabase를 이용해 text-to-sql를 테스트하고 추천 결과 확인

In [ ]:
!pip install langchain langchain_community sqlalchemy psycopg2 langchain_experimental openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 601.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from sqlalchemy import create_engine
from langchain_community.llms import OpenAI
import openai
from langchain.prompts import PromptTemplate

# Database credentials
dbname = ""
user = ""
password = ""
host = ""
port = ""

## Docs
[SQLDatabase](https://python.langchain.com/api_reference/community/utilities/langchain_community.utilities.sql_database.SQLDatabase.html#langchain_community.utilities.sql_database.SQLDatabase)

[SQLDatabaseChain](https://python.langchain.com/api_reference/experimental/sql/langchain_experimental.sql.base.SQLDatabaseChain.html#langchain_experimental.sql.base.SQLDatabaseChain)

In [ ]:
# Create an SQLAlchemy engine
connection_str = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}"
engine = create_engine(connection_str)

# Initialize the SQLDatabase object
sql_db = SQLDatabase(engine)

In [ ]:
print(sql_db.get_table_info())

프롬프트 참고
https://python.langchain.com/docs/tutorials/sql_qa/#system-prompt

In [ ]:
template = """
사용자의 요구사항에 따라 적절하게 데이터베이스 sql을 생성하고 필요하면 여러 번의 sql을 통해 정보를 얻어오세요. 그 정보를 토대로 사용자에게 답변하세요. influencer 테이블의 nickname 값의 리스트로 알려줘야 합니다.
평균 좋아요 수, 평균 댓글 수, 팔로워 수는 meta 테이블에 있습니다. 미디어의 종류는 "피드", "릴스"로 나뉘며 media 테이블의 media_product_type이 각각 'FEED', 'REELS'입니다.
다음은 sql의 예시입니다.
특정 인플루언서의 최신 팔로워 수를 얻고 싶을 때: "select follower_cnt from meta where influencer_id=(select influencer_id from influencer where nickname='influencer_nickname') order by created_at desc limit(1)"
'틴트'란 키워드를 게시글에 쓴 인플루언서를 찾고 싶을 때: "select nickname from influencer where influencer_id in (select influencer_id from media where media_id in (select media_id from media_hash_tag where hash_tag_id in (select hash_tag_id from hash_tag where name='틴트')))"
DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
사용자의 채팅: {chat_content}
"""
prompt = PromptTemplate(
    input_variables=["chat_content"],
    template=template
)

user_chat = "팔로워가 만 명 이상인 인플루언서를 모두 알려줘."

formatted_prompt = prompt.format(chat_content=user_chat)

print(formatted_prompt)

In [ ]:
# Define the LLM from OpenAI (GPT-3.5 or GPT-4)
llm = OpenAI(
      model="gpt-3.5-turbo-instruct",
      temperature=0,
      max_retries=2,
      api_key="" # api key
    )

# Use SQLDatabaseChain to run queries
db_chain = SQLDatabaseChain.from_llm(llm, sql_db)

# Example query
result = db_chain.invoke(formatted_prompt)
print(result['result'])